In [65]:
!pip install -r requirements.txt -q

You should consider upgrading via the 'E:\ThS\HK241\MathFundamental\Thay_Tuan_Anh\MaxFlowProblem\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [52]:
import requests
import time

import osmnx as ox
import networkx as nx
import numpy as np

In [261]:
base_coords = [
    (10.801522348071677, 106.71143076476699),  # Nga tu Hang Xanh
    (10.81339, 106.66519),  # San bay Tan Son Nhat
    (10.7928977, 106.653385),  # Nga tu Bay Hien
]

BASE_OSM_IDS = [
    971692819,
    54242195,
    11468863430,
]

In [ ]:
def geocode_osm(query):
    url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://nominatim.openstreetmap.org'
    }

    retries = 0

    while True:
        response = requests.get(url, headers=headers)
        time.sleep(1)
        
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Failed to reverse geocode. Status code: {response.status_code}. Retrying...")
            retries += 1
            if retries >= 3:
                return "Not found"

### Get nodes including base nodes

In [54]:
bbox = [10.817339, 10.793797, 106.711431, 106.651829]
G = ox.graph_from_bbox(bbox=bbox, network_type="drive")
nodes = list(G.nodes)
len(nodes), nodes[0]

C:\Users\Asus\AppData\Local\Temp\ipykernel_2856\3773831754.py:2: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = ox.graph_from_bbox(bbox=bbox, network_type="drive")


(1892, 366367322)

### Count the number of ones in adjacency matrix -> The number of edges

In [ ]:
# sampled_nodes = np.random.choice(nodes, 200, replace=False)
# subgraph = G.subgraph(sampled_nodes)
adj_matrix = nx.to_numpy_array(G)
adj_matrix.shape

(1892, 1892)

In [60]:
adj_matrix.sum(axis=0)

array([3., 4., 1., ..., 1., 2., 2.])

In [56]:
adj_matrix.sum()

4116.0

In [57]:
from pathlib import Path 

save_file = Path('results/adj_matrix.npy')
save_file.parent.mkdir(parents=True, exist_ok=True)
np.save(save_file, adj_matrix)

In [61]:
node_data = [{"node_id": node, "lat": data['y'], "lon": data['x']} for node, data in G.nodes(data=True)]
node_data[0]

{'node_id': 366367322, 'lat': 10.7991329, 'lon': 106.6571621}

In [66]:
def reverse_geocode_osm(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://nominatim.openstreetmap.org'
    }

    retries = 0

    while True:
        response = requests.get(url, headers=headers)
        time.sleep(1)
        
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Failed to reverse geocode. Status code: {response.status_code}. Retrying...")
            retries += 1
            if retries >= 3:
                return "Not found"

In [67]:
data = [None] * len(node_data) 

In [68]:
from tqdm import tqdm

print("Get data ...")

for i, node_object in enumerate(tqdm(node_data)):
    data[i] = reverse_geocode_osm(node_object['lat'], node_object['lon'])

Get data ...


  0%|          | 0/1892 [00:00<?, ?it/s]

100%|██████████| 1892/1892 [58:40<00:00,  1.86s/it] 


In [70]:
from file import dump_json, load_json

dump_json('results/node_data.json', data)

In [81]:
base_data = [reverse_geocode_osm(i[0], i[1]) for i in base_coords]
[x["display_name"] for x in base_data]

['Hang Xanh Overpass, Ward 25, Binh Thanh District, Ho Chi Minh City, 72508, Vietnam',
 'International Terminal, International Terminal Arrival Drive, Ward 2, Tan Binh District, Ho Chi Minh City, 72100, Vietnam',
 'Ward 4, Tan Binh District, Ho Chi Minh City, 72106, Vietnam']

In [90]:
full_data = data 
len(full_data) 

1892

### Display data

In [91]:
import pandas as pd

df = pd.DataFrame(full_data)
df.head()

,place_id,licence,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,address,boundingbox
0,237475900,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,668267736,10.7991329,106.6571621,highway,residential,26,0.053400,road,Hẻm 97 Nguyễn Thái Bình,"Hẻm 97 Nguyễn Thái Bình, Ward 4, Tan Binh Dist...","{'road': 'Hẻm 97 Nguyễn Thái Bình', 'quarter':...","[10.7987347, 10.7991329, 106.6571580, 106.6573..."
1,237823328,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,32581480,10.8102713,106.6668589,highway,residential,26,0.053400,road,Cửu Long,"Cửu Long, Ward 2, Tan Binh District, Ho Chi Mi...","{'road': 'Cửu Long', 'quarter': 'Ward 2', 'sub...","[10.8094886, 10.8108099, 106.6648221, 106.6682..."
2,237620675,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,806265451,10.7965385,106.6525594,highway,tertiary,26,0.053400,road,Nguyễn Thái Bình,"Nguyễn Thái Bình, Ward 12, Tan Binh District, ...","{'road': 'Nguyễn Thái Bình', 'quarter': 'Ward ...","[10.7951914, 10.7965385, 106.6504457, 106.6525..."
3,237620335,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,4209332089,10.7961796,106.6643062,amenity,pharmacy,30,0.000067,amenity,Diệt Mối Phúc Anh,"Diệt Mối Phúc Anh, 39, Pham Van Hai, Ward 1, T...","{'amenity': 'Diệt Mối Phúc Anh', 'house_number...","[10.7961296, 10.7962296, 106.6642562, 106.6643..."
4,239485905,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,503314829,10.8011312,106.6757184,highway,residential,26,0.053400,road,Hẻm 17 Hồ Văn Huê,"Hẻm 17 Hồ Văn Huê, Ward 9, Phu Nhuan District,...","{'road': 'Hẻm 17 Hồ Văn Huê', 'quarter': 'Ward...","[10.8011312, 10.8017269, 106.6748503, 106.6757..."


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892 entries, 0 to 1891
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   place_id      1892 non-null   int64  
 1   licence       1892 non-null   object 
 2   osm_type      1892 non-null   object 
 3   osm_id        1892 non-null   int64  
 4   lat           1892 non-null   object 
 5   lon           1892 non-null   object 
 6   class         1892 non-null   object 
 7   type          1892 non-null   object 
 8   place_rank    1892 non-null   int64  
 9   importance    1892 non-null   float64
 10  addresstype   1892 non-null   object 
 11  name          1892 non-null   object 
 12  display_name  1892 non-null   object 
 13  address       1892 non-null   object 
 14  boundingbox   1892 non-null   object 
dtypes: float64(1), int64(3), object(11)
memory usage: 221.8+ KB


### Adjacency matrix

In [88]:
adj_matrix = np.load("results\\adj_matrix.npy")
adj_matrix.shape

(1892, 1892)

In [94]:
adj_matrix[-10:, -10:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [95]:
np.sum(adj_matrix)

4116.0

In [96]:
adj_matrix[adj_matrix == 1]

array([1., 1., 1., ..., 1., 1., 1.])

In [98]:
np.array([
    [0,10,0,0,0],
    [1555,0,15,0,0],
    [0,0,0,0,0],
    [0,0,80,0,9],
    [0,0,0,0,0],
])

array([[   0,   10,    0,    0,    0],
       [1555,    0,   15,    0,    0],
       [   0,    0,    0,    0,    0],
       [   0,    0,   80,    0,    9],
       [   0,    0,    0,    0,    0]])

In [ ]:
lat1, lon1 = nodes[0]
lat2, lon2 = nodes[1]
n = 10 

### Search for base nodes and "Ngã ba, ngã tư, ..."

In [170]:
airport_query = "Tân Sơn Nhất Thành phố Hồ Chí Minh, Việt Nam"
hangxanh_query = "ngã tư hàng xanh Thành phố Hồ Chí Minh, Việt Nam"
bayhien_query = "ngã tư bảy hiền Thành phố Hồ Chí Minh, Việt Nam"

interested_keywords = [
    "ngã ba", 
    "ngã tư", 
    "nút giao", 
    "vòng xoay", 
    "ngã năm", 
    "ngã sáu", 
    "bùng binh",
    "công trường dân chủ",
    "giao lộ",
    "trường đại học",
]
interested_queries = [x + " Thành phố Hồ Chí Minh, Việt Nam" for x in interested_keywords]

In [171]:
def search_osm(**params):
    url = f"https://nominatim.openstreetmap.org/search"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'vi-VN,vi;q=0.9' ,
        'Referer': 'https://nominatim.openstreetmap.org'
    }

    retries = 0

    while True:
        response = requests.get(url, params=params, headers=headers)
        time.sleep(1)
        
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Search failed. Status code: {response.status_code}. Retrying...")
            retries += 1
            if retries >= 3:
                return "Not found"

In [172]:
def dedupe_list_by_key(input_list, key):
    seen = set()
    result = []
    
    for obj in input_list:
        key_value = obj[key]
        if key_value not in seen:
            seen.add(key_value)
            result.append(obj)
    
    return result

In [173]:
def get_data(query, kpi=10):
    data = []

    while len(data) < kpi:
        params = {
            "q": query,
            "format": "json",
            "limit": kpi,
        }

        batch_data = search_osm(**params)
        data.extend(batch_data)

    data = dedupe_list_by_key(data, "osm_id")
    return data

In [174]:
airport_data = get_data(airport_query)
len(airport_data)

7

In [175]:
filtered_airport_data = [airport for airport in airport_data if "Tân Sơn Nhất" in airport['name']]
len(filtered_airport_data)

6

In [176]:
hangxanh_data = get_data(hangxanh_query)
len(hangxanh_data)

5

In [177]:
bayhien_data = get_data(bayhien_query)
len(bayhien_data)

2

In [178]:
additional_data = []

for keyword, query in zip(interested_keywords, interested_queries):
    _data = get_data(query, 100)
    _data = [_data for _data in _data if _data['name'].lower().startswith(keyword)]
    additional_data.extend(_data)

len(additional_data)

194

In [179]:
node_data = additional_data + airport_data + hangxanh_data + bayhien_data
node_data = dedupe_list_by_key(node_data, "osm_id")
len(node_data)

204

In [180]:
dump_json("results/node_data2.json", node_data)

## Load graph to calculate distance

In [181]:
import pickle

with open("results/graph.pkl", "rb") as f:
    graph = pickle.load(f)

In [189]:
list(graph.nodes(data=True))[0]

(366225951, {'y': 10.6277126, 'x': 106.7368656, 'street_count': 3})

In [247]:
intersect_points = [data for data in graph.nodes(data=True) if data[1]['street_count'] > 1]
len(intersect_points)

136834

In [257]:
import geopy.distance
from tqdm import tqdm

def get_k_closest_nodes(lat, lon, nodes, kpi=100):
    distances = []
    
    for node, data in tqdm(nodes):
        node_lat = data['y']
        node_lon = data['x']
        
        # Calculate the geographical distance between the base and graph node
        distance = geopy.distance.distance((lat, lon), (node_lat, node_lon)).km
        
        # Append the distance and the node ID
        distances.append((distance, node))
    
    # Sort the nodes by distance and return the nearest ones
    distances.sort(key=lambda x: x[0])  # Sort by distance (ascending)
    return [node for _, node in distances[:kpi]]  # Get the 200 closest nodes

In [258]:
hangxanh_closest_nodes = get_k_closest_nodes(*base_coords[0], intersect_points)
len(hangxanh_closest_nodes)

  0%|          | 0/136834 [00:00<?, ?it/s]

100%|██████████| 136834/136834 [00:59<00:00, 2296.23it/s]


100

In [260]:
graph.nodes()[hangxanh_closest_nodes[0]]

{'y': 10.8016017, 'x': 106.7115512, 'street_count': 3}

In [266]:
base_coords[1] = (10.81245, 106.66506)
tansonnhat_closest_nodes = get_k_closest_nodes(*base_coords[1], intersect_points)
len(tansonnhat_closest_nodes)

100%|██████████| 136834/136834 [01:00<00:00, 2262.08it/s]


100

In [267]:
graph.nodes()[tansonnhat_closest_nodes[0]]

{'y': 10.8124437, 'x': 106.6651622, 'street_count': 3}

In [ ]:
bayhien_closest_nodes = get_k_closest_nodes(*base_coords[2], intersect_points)
len(bayhien_closest_nodes)

100%|██████████| 136834/136834 [01:01<00:00, 2234.64it/s]


100

In [265]:
graph.nodes()[bayhien_closest_nodes[0]]

{'y': 10.7929415, 'x': 106.6533454, 'street_count': 3}

In [305]:
all_nodes = set(hangxanh_closest_nodes + tansonnhat_closest_nodes + bayhien_closest_nodes)
all_nodes = list(all_nodes)
len(all_nodes)

300

In [297]:
adj_matrix = nx.to_numpy_array(graph.subgraph(all_nodes))
adj_matrix = (adj_matrix > 0).astype(int)
adj_matrix.sum()

561

In [298]:
edges = pd.Series(adj_matrix.ravel())
edges.value_counts()

0    89439
1      561
Name: count, dtype: int64

In [309]:
def get_node_data(node):
    data = graph.nodes(data=True)[node]
    return {
        "node_id": node,
        "lat": data['y'],
        "lon": data['x'],
    }

node_data3 = [get_node_data(node) for node in all_nodes]
dump_json("results/node_data3.json", node_data3)

In [310]:
def est_capacity(adj_value, row, col, node_data=node_data3, est_width=20): 
    if int(adj_value) == 0:
        return 0
    
    row_node = node_data[row]
    col_node = node_data[col]
    row_lat, row_lon = row_node['lat'], row_node['lon']
    col_lat, col_lon = col_node['lat'], col_node['lon']
    distance = geopy.distance.distance((row_lat, row_lon), (col_lat, col_lon)).m
    capacity = adj_value * distance * est_width
    if capacity <= 0:
        print(adj_value, distance)
    return capacity

In [311]:
updated_adj_matrix = np.vectorize(est_capacity)(adj_matrix, np.indices(adj_matrix.shape)[0], np.indices(adj_matrix.shape)[1])
updated_adj_matrix.shape

1 0.0
1 0.0


(300, 300)

In [312]:
capacities = pd.Series(updated_adj_matrix.ravel())
capacities.describe()

count     90000.000000
mean        271.035833
std        5206.882448
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      134019.000000
dtype: float64

In [313]:
len(capacities[capacities > 0])

559

In [314]:
np.save("results/adj_matrix3.npy", updated_adj_matrix)

## Load data and save edges data

In [5]:
from file import load_json, dump_json
import numpy as np

adj_matrix = np.load("results/adj_matrix3.npy")
node_data = load_json("results/node_data3.json")

print(adj_matrix.shape, len(node_data))

(300, 300) 300


In [9]:
n = len(node_data)
edges = []

for i in range(n):
    node_i = node_data[i]
    for j in range(n):
        node_j = node_data[j]
        if adj_matrix[i, j] > 0:
            edges.append({
                "src": node_i['node_id'],
                "dst": node_j['node_id'],
                "capacity": int(adj_matrix[i, j]),
            })

len(edges)

559

In [10]:
dump_json("results/edge_data.json", edges)